In [1]:
import pandas as pd

In [2]:
path = 'data/yearly_full_release_long_format.csv'

In [4]:
data = pd.read_csv(path)

In [5]:
data

,Area,Country code,Year,Area type,Continent,Ember region,EU,OECD,G20,G7,Category,Subcategory,Variable,Unit,Value,YoY absolute change,YoY % change
0,Afghanistan,AFG,2000,Country,Asia,Asia,0.0,0.0,0.0,0.0,Capacity,Aggregate fuel,Clean,GW,0.19,NaN,NaN
1,Afghanistan,AFG,2000,Country,Asia,Asia,0.0,0.0,0.0,0.0,Capacity,Aggregate fuel,Fossil,GW,0.03,NaN,NaN
2,Afghanistan,AFG,2000,Country,Asia,Asia,0.0,0.0,0.0,0.0,Capacity,Aggregate fuel,Gas and Other Fossil,GW,0.03,NaN,NaN
3,Afghanistan,AFG,2000,Country,Asia,Asia,0.0,0.0,0.0,0.0,Capacity,Aggregate fuel,"Hydro, Bioenergy and Other Renewables",GW,0.19,NaN,NaN
4,Afghanistan,AFG,2000,Country,Asia,Asia,0.0,0.0,0.0,0.0,Capacity,Aggregate fuel,Renewables,GW,0.19,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
316535,Zimbabwe,ZWE,2020,Country,Africa,Africa,0.0,0.0,0.0,0.0,Power sector emissions,Fuel,Other Fossil,mtCO2,0.03,0.00,0.00
316536,Zimbabwe,ZWE,2020,Country,Africa,Africa,0.0,0.0,0.0,0.0,Power sector emissions,Fuel,Other Renewables,mtCO2,0.00,0.00,NaN
316537,Zimbabwe,ZWE,2020,Country,Africa,Africa,0.0,0.0,0.0,0.0,Power sector emissions,Fuel,Solar,mtCO2,0.00,0.00,NaN
316538,Zimbabwe,ZWE,2020,Country,Africa,Africa,0.0,0.0,0.0,0.0,Power sector emissions,Fuel,Wind,mtCO2,0.00,0.00,NaN


In [11]:
data['Category'].unique()

array(['Capacity', 'Electricity demand', 'Electricity generation',
       'Electricity imports', 'Power sector emissions'], dtype=object)

In [20]:
data_clean = \
data[(data['Year']==2021) & \
     (data['Category']=='Electricity generation') & \
     (data['Unit']=='%') & \
     (data['Subcategory']=='Aggregate fuel')][['Area','Variable','Value']].set_index(['Area','Variable'])

In [22]:
data_clean.unstack().droplevel

Value                              \
Variable                  Clean Fossil Gas and Other Fossil   
Area                                                          
Africa                    24.46  75.53                48.30   
Argentina                 32.35  67.65                65.64   
Armenia                   59.69  40.32                40.32   
Asia                      30.69  69.32                11.83   
Australia                 29.04  70.97                19.66   
...                         ...    ...                  ...   
United Kingdom            55.04  44.95                43.02   
United States of America  39.66  60.35                38.73   
Uruguay                   75.54  24.47                24.47   
Viet Nam                  37.12  62.87                11.59   
World                     38.44  61.55                25.00   

                                                                           \
Variable                 Hydro, Bioenergy and Other Renewables Renewables   
Area                                                                        
Africa                                                   19.20      22.97   
Argentina                                                14.92      25.35   
Armenia                                                  24.57      24.84   
Asia                                                     15.92      25.50   
Australia                                                 8.08      29.04   
...                                                        ...        ...   
United Kingdom                                           14.55      39.78   
United States of America                                  7.86      20.94   
Uruguay                                                  40.09      75.54   
Viet Nam                                                 25.67      37.12   
World                                                    18.06      28.49   

                                         
Variable                 Wind and Solar  
Area                                     
Africa                             3.77  
Argentina                         10.43  
Armenia                            0.27  
Asia                               9.58  
Australia                         20.96  
...                                 ...  
United Kingdom                    25.23  
United States of America          13.08  
Uruguay                           35.45  
Viet Nam                          11.45  
World                             10.43  

[89 rows x 6 columns]